In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline
import escher
#import cobra.test
from cobra.flux_analysis.phenotype_phase_plane import production_envelope, add_envelope
from cobra.flux_analysis import flux_variability_analysis


In [2]:
from cobra.io import read_sbml_model
model = read_sbml_model('iML1515.xml')

In [3]:
print(model.objective)

Maximize
1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685


In [4]:
#model.optimize().objective_value

In [5]:
#Output solution
print("Max. BM production [mmol gDW^-1 h^-1]:", model.optimize().objective_value)
print("Max. L-serine production [mmol gDW^-1 h^-1]:", model.reactions.EX_ser__L_e.flux)
print("Theoretical max. yield [mmol_L-ser / mmol_glc]:", model.reactions.EX_ser__L_e.flux / (-1*model.reactions.EX_glc__D_e.flux))

Max. BM production [mmol gDW^-1 h^-1]: 0.8769972144269618
Max. L-serine production [mmol gDW^-1 h^-1]: 0.0
Theoretical max. yield [mmol_L-ser / mmol_glc]: 0.0


In [6]:
with model:
    model.objective = model.reactions.EX_ser__L_e
    model.optimize();
#Output solution
#print("Max. BM production [mmol gDW^-1 h^-1]:", model.optimize().objective_value)
    print("Max. L-serine production [mmol gDW^-1 h^-1]:", model.reactions.EX_ser__L_e.flux)
    print("Theoretical max. yield [mmol_L-ser / mmol_glc]:", model.reactions.EX_ser__L_e.flux / (-1*model.reactions.EX_glc__D_e.flux))

Max. L-serine production [mmol gDW^-1 h^-1]: 20.625882352941172
Theoretical max. yield [mmol_L-ser / mmol_glc]: 2.0625882352941174


In [7]:
model.metabolites.ser__L_p.summary()

In [8]:
# Flux variability analysis of reactions to ser_L at 90% BM
reactions_OE = [model.reactions.PGCD, model.reactions.PSERT, model.reactions.PSP_L,model.reactions.SERAT, model.reactions.GHMT2r, 
                model.reactions.LSERDHr,model.reactions.SERD_L, model.reactions.TRPS2, model.reactions.GPDDA3pp, model.reactions.SERtpp,
                model.reactions.SERtex, model.reactions.EX_ser__L_e, model.reactions.BIOMASS_Ec_iML1515_core_75p37M,
               model.reactions.GLCt2pp]
fva = flux_variability_analysis(model, reaction_list = reactions_OE,
fraction_of_optimum=0.9)
print(fva)

                                 minimum    maximum
PGCD                            0.000000  24.670609
PSERT                           0.000000  15.865461
PSP_L                           0.000000  15.865461
SERAT                           0.195337   9.676144
GHMT2r                         -4.507400   9.466666
LSERDHr                         0.000000  14.513470
SERD_L                          0.000000  14.513470
TRPS2                           0.000000  14.513470
GPDDA3pp                        0.000000   0.000000
SERtpp                          0.000000  93.256000
SERtex                         -2.168744   0.000000
EX_ser__L_e                     0.000000   2.168744
BIOMASS_Ec_iML1515_core_75p37M  0.789297   0.876997
GLCt2pp                         0.000000  10.000000


In [9]:
# Overexpression / Underexpression
# From FVA we can get 2.16 Ser with 90 % BM
# GLCt2pp changes the glucose uptake to bypass our pre-cursor

with model:
    
    #model.reactions.PGCD.lower_bound = 50
    #model.reactions.PSERT.lower_bound = 30
    #model.reactions.PSP_L.lower_bound = 30
    #model.reactions.SERtpp.lower_bound = 20
    #model.reactions.SERtex.lower_bound = -20
    #model.reactions.GPDDA3pp.bounds = 0.5,5
    
    #model.reactions.SERAT.bounds = 0,5           #Lowers
    #model.reactions.GHMT2r.bounds = -5,0
    #model.reactions.SERD_L.bounds = 0,1
    #model.reactions.TRPS2.bounds = 0,1
    
    #model.reactions.EX_ser__L_e.lower_bound = 5
    
    #model.reactions.GLCptspp.knock_out()          #GLCptspp  #
    #model.genes.b2551.knock_out()                  #GHMT2r  Lowers
    #model.genes.b1539.knock_out()                 #LSERDHr
    #model.genes.b1260.knock_out()                 #TRPS2    #Essential
    #model.genes.b3607.knock_out()                 #SERAT    #Essential
    
    model.reactions.EX_ser__L_e.bounds = 5,5
    model.optimize();
    
    print("Max. BM production [mmol gDW^-1 h^-1]:", model.optimize().objective_value, " || % of optimum:", model.optimize().objective_value/0.876997*100)
    print("Max. L-serine production [mmol gDW^-1 h^-1]:", model.reactions.EX_ser__L_e.flux)
    print("Theoretical max. yield [mmol_L-ser / mmol_glc]:", model.reactions.EX_ser__L_e.flux / (-1*model.reactions.EX_glc__D_e.flux))
    print(" ")
    print("Fluxes in pathway to Ser")
    print("PGCD :", model.reactions.PGCD.flux)
    print("PSERT :", model.reactions.PSERT.flux)
    print("PSP_L :", model.reactions.PSP_L.flux)
    print("GHMT2r :", model.reactions.GHMT2r.flux)
    print("SERAT :", model.reactions.SERAT.flux)
    print("Fluxes to and from 3pg_c (pre-cursor)")
    print("PGK :", model.reactions.PGK.flux)
    print("PGM :", model.reactions.PGM.flux)
    print("PGCD :", model.reactions.PGCD.flux, "To Ser")

Max. BM production [mmol gDW^-1 h^-1]: 0.6729299410413994  || % of optimum: 76.73115655371674
Max. L-serine production [mmol gDW^-1 h^-1]: 5.0
Theoretical max. yield [mmol_L-ser / mmol_glc]: 0.5
 
Fluxes in pathway to Ser
PGCD : 6.155982268803919
PSERT : 6.155982268803919
PSP_L : 6.155982268803919
GHMT2r : 0.7506752194548023
SERAT : 0.1665380476688077
Fluxes to and from 3pg_c (pre-cursor)
PGK : -16.70972075812051
PGM : -10.553738489316592
PGCD : 6.155982268803919 To Ser


In [10]:
#X = model.copy()
#X.reactions.EX_ser__L_e.bounds = 2,5
#X.reactions.GLCptspp.knock_out()
#X.optimize();
#X.metabolites.glc__D_e.summary()

In [11]:
#X.reactions.PGCD.lower_bound = 30
#X.reactions.PSERT.lower_bound = 20
#X.reactions.PSP_L.lower_bound = 20
#X.reactions.EX_ser__L_e.lower_bound = 1
#X.reactions.SERD_L.upper_bound = 10
#X.metabolites.ser__L_c.summary()

In [12]:
#model.objective = model.reactions.EX_ser__L_e
#model.optimize();
print("Max. BM production [mmol gDW^-1 h^-1]:", X.optimize().objective_value)
print("Max. L-serine production [mmol gDW^-1 h^-1]:", X.reactions.EX_ser__L_e.flux)
print("Theoretical max. yield [mmol_L-ser / mmol_glc]:", X.reactions.EX_ser__L_e.flux / (-1*X.reactions.EX_glc__D_e.flux))

NameError: name 'X' is not defined

In [ ]:
from cameo import load_model
model1 = load_model('iML1515.xml')

In [ ]:
from cameo import fba
fba_result = fba(model1)

In [ ]:
fba_result.display_on_map('E. coli coremetabolism_with Serine.json')

In [ ]:
# BM set to ~60% of optimum
with model1:
    model1.reactions.BIOMASS_Ec_iML1515_core_75p37M.bounds = 0.5,1000
    model1.objective = model1.reactions.EX_ser__L_e
    model1.optimize();
    qqq = fba(model1)

In [ ]:
qqq.display_on_map('E. coli coremetabolism_with Serine.json')

In [ ]:
from cameo import phenotypic_phase_plane

In [ ]:
QQ = model1.copy()

In [ ]:
#QQ.reactions.EX_ser__L_e.bounds = 1,1000
QQ.reactions.BIOMASS_Ec_iML1515_core_75p37M.bounds = 0.5,1000
QQ.objective = QQ.reactions.EX_ser__L_e
print("Ser prod.: ", QQ.optimize().objective_value)

In [ ]:
# Same result as the EX_ser value in FBA above
QQ.metabolites.ser__L_p.summary()

In [ ]:
from cameo import load_model
from cameo import phenotypic_phase_plane
from cameo.visualization.plotting.with_plotly import PlotlyPlotter
from cameo.visualization import plotting

In [ ]:
XYZ = model1.copy()

In [ ]:
XYZ.optimize()

In [ ]:
#production_envelope = phenotypic_phase_plane(XYZ, variables=[XYZ.reactions.BIOMASS_Ec_iML1515_core_75p37M], 
                                             objective=XYZ.reactions.SERtex)

In [ ]:
# Plot doesn't work
#production_envelope.plot()

In [ ]:
# The code below works but is very slow

In [ ]:
from cameo.strain_design.deterministic import DifferentialFVA

In [ ]:
#model1.reactions.EX_o2_e.lower_bound = 0
#reference_model = model1.copy()
#biomass_rxn = reference_model.reactions.BIOMASS_Ec_iML1515_core_75p37M
#biomass_rxn.lower_bound = 0.
#target = reference_model.reactions.EX_ser__L_e

In [ ]:
#diff_fva = DifferentialFVA(design_space_model=model,
                           reference_model=reference_model,
                           objective=target,
                           variables=[biomass_rxn],
                           normalize_ranges_by=biomass_rxn,
                           points=10)

In [ ]:
# Takes 7 min!

#%time result = diff_fva.run(surface_only=True)
#result.plot()

In [ ]:
#result

In [ ]:
#result.plot(5, variables=['EX_ser__L_e', 'GHMT2r', 'SERD_L', 'PSP_L'])

In [ ]:
#result.display_on_map(5, map_name="e_coli_core.Core metabolism")